N-gram model

In [1]:
# !pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

In [3]:
# creating placeholder for the model
model = defaultdict(lambda: defaultdict(lambda: 0))

#count frequency of co occurence
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right = True, pad_left = True):
        model[(w1,w2)][w3] += 1

#transform counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [4]:
dict(model["the","price"])

{'yesterday': 0.004651162790697674,
 'of': 0.3209302325581395,
 'it': 0.05581395348837209,
 'effect': 0.004651162790697674,
 'cut': 0.009302325581395349,
 'for': 0.05116279069767442,
 'paid': 0.013953488372093023,
 'to': 0.05581395348837209,
 'increases': 0.013953488372093023,
 'used': 0.004651162790697674,
 'climate': 0.004651162790697674,
 '.': 0.023255813953488372,
 'cuts': 0.009302325581395349,
 'reductions': 0.004651162790697674,
 'limit': 0.004651162790697674,
 'now': 0.004651162790697674,
 'moved': 0.004651162790697674,
 'per': 0.013953488372093023,
 'adjustments': 0.004651162790697674,
 '(': 0.009302325581395349,
 'slumped': 0.004651162790697674,
 'is': 0.018604651162790697,
 'move': 0.004651162790697674,
 'evolution': 0.004651162790697674,
 'differentials': 0.009302325581395349,
 'went': 0.004651162790697674,
 'the': 0.013953488372093023,
 'factor': 0.004651162790697674,
 'Royal': 0.004651162790697674,
 ',': 0.018604651162790697,
 'again': 0.004651162790697674,
 'changes': 0.0

In [6]:
import random

#starting words
text = ["today","the"]
sentence_finished = False

while not sentence_finished:
    # select random probability threshold
    r = random.random()
    accumulator = .0

    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
        #select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            break

    if text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in text if t]))

today the price differentials between the Bundesbank did not know how the figures were not comparable since the partnership said .


In [7]:
# !pip install keras

In [8]:
#!pip install tensorflow
# !pip install --upgrade setuptools

In [9]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
data = """Federated learning is a machine learning framework to train local models inside distributed servers and then aggregate to form a single generalizable global model. 
The entity that coordinates the training is called a central server. 
The central server initiates, aggregates the model weights and distributes among the servers. 
Decentralized federated learning frameworks do not have a central server for coordination. 
Every distributed server aggregates the model to lastly output a global generalized version.
Numerous literature have been published over the years related to federated learning.
Several machine learning and deep learning algorithms have been extensively tried and tested in this framework. 
As per the information available none of the studies have focused upon application of rule based machine learning algorithms in a federated learning framework. 
Rule based machine learning algorithms are easy to interpret and understand as they are based on simple logical statements. 
Privacy preserving technologies have become need of the hour due to rising instances of sensitive data leakages. 
Combining these three different concepts to form one single framework that works in uniformity is a challenging task that involves extensive knowledge gathering and implementation. 
The enthusiasm to take this challenge and fulfill with dedication by innovating a novel approach and sharing insights with scientific and AI community forms primary motivation for selection of ‘Federated Rule Learning’ as my Master Thesis topic.
Machine Learning has become a go to solution in recent years to solve complex tasks that cannot be easily solved with traditional methods. They are widely used in industrial, academical and professional settings. 
The process of applying complex mathematical algorithms that help to understand and analyse data is known as machine learning.
The background processes that these algorithms use cannot be easily determined and hence are often called as black boxes or non interpretable. 
This was not the case with algorithms that were developed during the initial stages of machine learning research. 
Many algorithms in those times were based upon simple concepts such as logic, probability and algebra. 
Some of these algorithms are still in use today such as regression, decision trees and naive bayes. 
Rule based machine learning algorithms are a part of this category. 
They are based upon the concepts of logical reasoning.
Data is the most important and integral part in a machine learning framework. 
Data can be in any form and format such as numerical, textual, audio, image, and others. 
It is gathered from various sources and hence have varied structure and storage formats.
Data has become the new oil and is very precious to train small and large models having varied applications. 
This also makes it quite vulnerable to be misused by malign entities.
Before diving deeper into the concepts of federated learning it is important to understand where is data generated from.
Data is constantly generated and can be gathered from anywhere and everywhere. """



In [11]:
import re

def text_preprocessor(text):
    new_str = text.lower()
    new_str = re.sub(r"'s\b","",new_str)
    new_str = re.sub("[^a-zA-Z]"," ",new_str)
    long_words = []
    for i in new_str.split():
        if len(i)>=3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_cleaned = text_preprocessor(data)

In [12]:
data_cleaned

'federated learning machine learning framework train local models inside distributed servers and then aggregate form single generalizable global model the entity that coordinates the training called central server the central server initiates aggregates the model weights and distributes among the servers decentralized federated learning frameworks not have central server for coordination every distributed server aggregates the model lastly output global generalized version numerous literature have been published over the years related federated learning several machine learning and deep learning algorithms have been extensively tried and tested this framework per the information available none the studies have focused upon application rule based machine learning algorithms federated learning framework rule based machine learning algorithms are easy interpret and understand they are based simple logical statements privacy preserving technologies have become need the hour due rising inst

In [13]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

sequences = create_seq(data_cleaned)

Total Sequences: 2796


In [14]:
sequences

['federated learning machine lear',
 'ederated learning machine learn',
 'derated learning machine learni',
 'erated learning machine learnin',
 'rated learning machine learning',
 'ated learning machine learning ',
 'ted learning machine learning f',
 'ed learning machine learning fr',
 'd learning machine learning fra',
 ' learning machine learning fram',
 'learning machine learning frame',
 'earning machine learning framew',
 'arning machine learning framewo',
 'rning machine learning framewor',
 'ning machine learning framework',
 'ing machine learning framework ',
 'ng machine learning framework t',
 'g machine learning framework tr',
 ' machine learning framework tra',
 'machine learning framework trai',
 'achine learning framework train',
 'chine learning framework train ',
 'hine learning framework train l',
 'ine learning framework train lo',
 'ne learning framework train loc',
 'e learning framework train loca',
 ' learning framework train local',
 'learning framework train l

In [15]:
chars = sorted(list(set(data_cleaned)))
mapping = dict((c,i) for i,c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encode_seq(sequences)

In [16]:
sequences

[[6,
  5,
  4,
  5,
  17,
  1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17],
 [5,
  4,
  5,
  17,
  1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17,
  13],
 [4,
  5,
  17,
  1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17,
  13,
  9],
 [5,
  17,
  1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13],
 [17,
  1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7],
 [1,
  19,
  5,
  4,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0,
  12,
  1,
  3,
  8,
  9,
  13,
  5,
  0,
  11,
  5,
  1,
  17,
  13,
  9,
  13,
  7,
  0

In [19]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [24]:
from sklearn.model_selection import train_test_split

vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: ', X_train.shape, '\nTest shape: ', X_test.shape)
print('Train shape: ', y_train.shape, '\nTest shape: ', y_test.shape)

Train shape:  (2236, 30) 
Test shape:  (560, 30)
Train shape:  (2236, 26) 
Test shape:  (560, 26)


In [26]:
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(15, recurrent_dropout=0.1, dropout=0.1))
# model.add(Flatten())
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

model.compile(loss = 'categorical_crossentropy', metrics = ['acc'], optimizer = 'adam')
print(X_train.shape)
print(y_train.shape)

model.fit(X_train, y_train, epochs = 100, verbose = 2, validation_data = (X_test, y_test))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
(2236, 30)
(2236, 26)
Epoch 1/100
70/70 - 11s - 163ms/step - acc: 0.1324 - loss: 3.1414 - val_acc: 0.1393 - val_loss: 2.9539
Epoch 2/100
70/70 - 2s - 24ms/step - acc: 0.1476 - loss: 2.8837 - val_acc: 0.1589 - val_loss: 2.8220
Epoch 3/100
70/70 - 2s - 23ms/step - acc: 0.1834 - loss: 2.7611 - val_acc: 0.1911 - val_loss: 2.7100
Epoch 4/100
70/70 - 1s - 19ms/step - acc: 0.2227 - loss: 2.6571 - val_acc: 0.2196 - val_loss: 2.6261
Epoch 5/100
70/70 - 2s - 23ms/step - acc: 0.2478 - loss: 2.5819 - val_acc: 0.2196 - val_loss: 2.5698
Epoch 6/100
70/70 - 1s - 17ms/step - acc: 0.2728 - loss: 2.5203 - val_acc: 0.2411 - val_loss: 2.5251
Epoch 7/100
70/70 - 1s - 18ms/step - acc: 0.2835 - loss: 2.4766 - val_acc: 0.2571 - val_loss: 2.4960
Epoch 8/100
70/70 - 1s - 19ms/step - acc: 0.2979 - loss: 2.4390 - val_acc: 0.2661 - val_loss: 2.4725
Epoch 9/100
70/70 - 2s - 27ms/step - acc: 0.3068 - loss: 2.4084 - val_acc: 0.2571 - val_loss: 2.4488
Epoch 10/100
70/70 - 2s - 23ms/step - acc: 0.3086 - loss: 2.37

In [39]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # yhat = model.predict(encoded, verbose = 0)
        yhat = model.predict(encoded)
        idx = np.argmax(yhat)
        out_char = [i for i,index in mapping.items() if index == idx][0]
        # for char, index in mapping.items():
        #     if index == yhat:
        #         out_char = char
                # break
        in_text += out_char
    return in_text

In [40]:
inp = 'differential privacy'
print(len(inp))
print(generate_seq(model, mapping, 30, inp.lower(), 15))

20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
differential privacy the the the th
